# 利用cvxpy库求解整数规划

In [4]:
import cvxpy as cp
import numpy as np

c = np.array([40,90])  #定义目标变量
a = np.array([[9,7],[-7,-20]])  #定义约束矩阵
b = np.array([56,-70])  #定义约束条件的右边向量
x = cp.Variable(2,integer=True)  #定义两个整数决策变量  integer=True：指定变量为整数类型（默认 False 表示连续变量）；boolean=True：0-1 整数变量（等价于 integer=True 加边界 [0,1]）
obj = cp.Minimize(c@x)  #构造目标函数
cons = [a@x<=b, x>=0]  #构造约束条件
prob = cp.Problem(obj, cons)  #构建问题模型
prob.solve(solver='GLPK_MI', verbose=True)  #求解问题 verbose=True：打印求解过程的详细日志（迭代次数、目标值变化等）
print('最优值为:', prob.value)
print('最优解为:\n', x.value)

(CVXPY) Aug 04 07:48:25 PM: Your problem has 2 variables, 4 constraints, and 0 parameters.
(CVXPY) Aug 04 07:48:25 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 04 07:48:25 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 04 07:48:25 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 04 07:48:25 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Aug 04 07:48:25 PM: Compiling problem (target solver=GLPK_MI).
(CVXPY) Aug 04 07:48:25 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> GLPK_MI
(CVXPY) Aug 04 07:48:25 PM: Applying reduction Dcp2Cone
(CVXPY) Aug 04 07:48:25 PM: Applying reduction CvxAttr2Constr
(CVXPY) Aug 04 07:48:25 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Aug 04 07:48:25 PM: Applying reduction GLPK_MI
(CVXPY) Aug 04 07:48:25 PM: Finished problem compilation (

                                     CVXPY                                     
                                     v1.7.0                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------
最优值为: 350.0
最优解为:
 [2. 3.]


# 指派问题
1. 标准指派模型

一般提法是：拟分派n个人$A_1,A_2,...,A_n$去完成n项工作$B_1,B_2,...,B_n$，要求每项工作需一个人去完成，每个人需完成且仅需完成一项工作。已知$A_i$完成工作$B_i$的时间或费用等成本型指标值为$c_{ij}$，则应如何指派才能使总的工作效率最高？

In [6]:
import cvxpy as cp
import numpy as np

c = np.array([[4,8,7,15,12],
              [7,9,17,14,10],
              [6,9,12,8,7],
              [6,7,14,6,10],
              [6,9,12,10,6]])
x = cp.Variable((5,5), integer=True)
obj = cp.Minimize(cp.sum(cp.multiply(c,x)))  #cp.multiply(c, x) 表示对向量或矩阵 c 和 x 进行 逐元素乘法
cons = [0<=x,x<=1, cp.sum(x, axis=0, keepdims=True)==1, cp.sum(x, axis=1, keepdims=True)==1]
"""
keepdims=True:
    求和后保留原始维度，将求和结果的形状从 (m,n) 变为 (1,n)(axis=0)或 (m,1)(axis=1)。
    用途:维持矩阵形式,便于后续广播(Broadcasting)操作或与其他矩阵运算。
"""
prob = cp.Problem(obj, cons)
prob.solve(solver='GLPK_MI')
print('最优值为:', prob.value)
print('最优解为:\n', x.value)

最优值为: 34.0
最优解为:
 [[0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]]


# 装箱问题

1. 装箱总厚度(长度)最大模型

In [41]:
import cvxpy as cp
import numpy as np

L = np.array([48.7,52.0,61.3,72.0,48.7,52.0,64.0])
w = np.array([2000,3000,1000,500,4000,2000,1000])
a = np.array([8,7,9,6,6,4,8])
x = cp.Variable((2,7), integer=True)
obj = cp.Maximize(cp.sum(cp.multiply(x,L)))
con = [cp.sum(x, axis=0, keepdims=True)<=a.reshape(1,7),
       cp.sum(cp.multiply(x,L),axis=1,keepdims=True)<=1020, cp.sum(cp.multiply(x,w),axis=1,keepdims=True)<=40000,
       cp.sum(cp.multiply(x[:,4:],L[4:]))<=302.7, x>=0]
prob = cp.Problem(obj, con)
prob.solve(solver='GLPK_MI', verbose=True)
print('最优值为:', prob.value)
print('最优解为:\n', x.value)

(CVXPY) Aug 04 09:46:50 PM: Your problem has 14 variables, 26 constraints, and 0 parameters.
(CVXPY) Aug 04 09:46:50 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Aug 04 09:46:50 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Aug 04 09:46:50 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Aug 04 09:46:50 PM: Your problem is compiled with the CPP canonicalization backend.
(CVXPY) Aug 04 09:46:50 PM: Compiling problem (target solver=GLPK_MI).
(CVXPY) Aug 04 09:46:50 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> GLPK_MI
(CVXPY) Aug 04 09:46:50 PM: Applying reduction FlipObjective
(CVXPY) Aug 04 09:46:50 PM: Applying reduction Dcp2Cone
(CVXPY) Aug 04 09:46:50 PM: Applying reduction CvxAttr2Constr
(CVXPY) Aug 04 09:46:50 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Aug 04 09:46:50 PM: Apply

                                     CVXPY                                     
                                     v1.7.0                                    
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
-------------------------------------------------------------------------------
                                    Summary                                    
-------------------------------------------------------------------------------


(CVXPY) Aug 04 09:46:50 PM: Problem status: optimal
(CVXPY) Aug 04 09:46:50 PM: Optimal value: 2.039e+03
(CVXPY) Aug 04 09:46:50 PM: Compilation took 7.217e-03 seconds
(CVXPY) Aug 04 09:46:50 PM: Solver (including time spent in interface) took 4.297e-01 seconds


最优值为: 2039.4
最优解为:
 [[4. 1. 5. 3. 3. 2. 0.]
 [4. 6. 4. 3. 0. 1. 0.]]


2. 装箱总重量最大模型

同上